In [23]:
import cv2
import os 

import tensorflow as tf 
import tensorflow_datasets as tfds 
import PIL

import matplotlib.pyplot as plt

# Tutte le immagini saranno di grandezza 160x160
IMG_SIZE  = 160

def format_image(image , label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [26]:
builder = tfds.folder_dataset.ImageFolder('Dataset/')
raw_dataset= builder.as_dataset(split='Train', as_supervised = True , shuffle_files=True)

split_index = int(len(raw_dataset)*0.8)

raw_train = raw_dataset.take(split_index)       #Restituisce un dataset costituito da i primi "split_index" elementi di raw_dataset
raw_validation = raw_dataset.skip(split_index)  #Salta i primi "split_index" elementi di raw_dataset e restituisce un dataset costituito da gli elementi rimanenti

imageNumberList = [0 for i in range(0,43)]

print("Immagini dataset training: {}  ".format(len(raw_train)) )
print("Immagini dataset validazione: {} ".format(len(raw_validation)))

train = raw_train.map(format_image)
validation = raw_validation.map(format_image)

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False , weights='imagenet' , input_shape=IMG_SHAPE)
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
middle_layer = tf.keras.layers.Dense(units=120 , activation='relu' )
prediction_layer = tf.keras.layers.Dense(units=43 , activation='softmax')

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  middle_layer,
  prediction_layer
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

Immagini dataset training: 31367  
Immagini dataset validazione: 7842 


In [27]:
initial_epochs = 3

In [28]:
history = model.fit(train_batches, epochs=initial_epochs, validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

Epoch 1/3
981/981 [==============================] - 947s 960ms/step - loss: 0.6815 - accuracy: 0.7909 - val_loss: 0.2648 - val_accuracy: 0.9190
Epoch 2/3
981/981 [==============================] - 889s 905ms/step - loss: 0.2456 - accuracy: 0.9204 - val_loss: 0.1880 - val_accuracy: 0.9393
Epoch 3/3
981/981 [==============================] - 876s 893ms/step - loss: 0.1768 - accuracy: 0.9412 - val_loss: 0.1261 - val_accuracy: 0.9574
[0.7909267544746399, 0.9203940629959106, 0.9411802291870117]


In [29]:
model.save("RoadSignRecognition.h5") 